In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import svm



from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, classification_report

%store -r X y
X

,loan_amnt,term,int_rate,installment,annual_inc,issue_d,dti,open_acc,revol_bal,pub_rec_bankruptcies,...,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding
0,5000.0,36.0,10.65,162.87,24000.0,734472,27.65,3.0,13648.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,2500.0,60.0,15.27,59.83,30000.0,734472,1.00,3.0,1687.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,2400.0,36.0,15.96,84.33,12252.0,734472,8.72,2.0,2956.0,0.0,...,0,0,0,0,0,0,0,1,0,0
3,10000.0,36.0,13.49,339.31,49200.0,734472,20.00,10.0,5598.0,0.0,...,0,0,0,0,0,1,0,0,0,0
4,3000.0,60.0,12.69,67.79,80000.0,734472,17.94,15.0,27783.0,0.0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42531,3500.0,36.0,10.28,113.39,180000.0,732828,10.00,9.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,0
42532,1000.0,36.0,9.64,32.11,12000.0,732828,10.00,9.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,0
42533,2525.0,36.0,9.33,80.69,110000.0,732828,10.00,9.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,0
42534,6500.0,36.0,8.38,204.84,59000.0,732828,4.00,9.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,0


In [3]:
##### Funktion scor


def score_print(y,y_pred):
    
    cm = confusion_matrix(y,y_pred)
    print(cm)
    cp= classification_report(y,y_pred)
    print(cp)
    

In [4]:
###### Naiv-estimator


# TODO: Total number of records
n_records = len(X)

# TODO: Number of records where individual's income is more than $50,000
fully_paid = sum(y==1)

# TODO: Number of records where individual's income is at most $50,000
charged_off = n_records - fully_paid

# TODO: Percentage of individuals whose income is more than $50,000
greater_percent = 100 * fully_paid / n_records

# Print the results
print ("Total number of records: {}".format(n_records))
print ("Individuals has paid her credit fully: {}".format(fully_paid))
print ("Individuals hasn't paid her credit: {}".format(charged_off))
print ("Percentage of individuals that paid ther credit: {:.2f}%".format(greater_percent))

Total number of records: 42535
Individuals has paid her credit fully: 6422
Individuals hasn't paid her credit: 36113
Percentage of individuals that paid ther credit: 15.10%


In [5]:
#Split into train and test

X_train, X_test, y_train, y_test= train_test_split(X, y, test_size = 0.4, random_state = 70, stratify= y)

# Show the results of the split
print ("Training set has {} samples.".format(X_train.shape[0]))
print ("Testing set has {} samples.".format(X_test.shape[0]))

Training set has 25521 samples.
Testing set has 17014 samples.


In [6]:
numerical = [col for col in X if X.dtypes[col] == "float64"]
numerical

['loan_amnt',
 'term',
 'int_rate',
 'installment',
 'annual_inc',
 'dti',
 'open_acc',
 'revol_bal',
 'pub_rec_bankruptcies']

In [7]:
#### skalieren
from sklearn.preprocessing import MinMaxScaler


# Initialize a scaler, then apply it to the features
scaler = MinMaxScaler()
#numerical = ['loan_amnt', 'term', 'int_rate', 'installment', 'annual_inc',"issue_d","dti","open_acc","revol_bal","pub_rec_bankruptcies" ]
numerical = [col for col in X if X.dtypes[col] == "float64"]
X_train.loc[:,numerical] = scaler.fit_transform(X_train[numerical])
X_test.loc[:,numerical] = scaler.transform(X_test[numerical])



/Users/kli/neuefische/13_Decision_trees/ds-ensemble-methods/.venv/lib/python3.8/site-packages/pandas/core/indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
/Users/kli/neuefische/13_Decision_trees/ds-ensemble-methods/.venv/lib/python3.8/site-packages/pandas/core/indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)


In [8]:
X_train.isna().sum()

loan_amnt                              0
term                                   0
int_rate                               0
installment                            0
annual_inc                             0
issue_d                                0
dti                                    0
open_acc                               0
revol_bal                              0
pub_rec_bankruptcies                   0
emp_length_10+ years                   0
emp_length_2 years                     0
emp_length_3 years                     0
emp_length_4 years                     0
emp_length_5 years                     0
emp_length_6 years                     0
emp_length_7 years                     0
emp_length_8 years                     0
emp_length_9 years                     0
emp_length_< 1 year                    0
emp_length_no info at all              0
home_ownership_NONE                    0
home_ownership_OTHER                   0
home_ownership_OWN                     0
home_ownership_R

In [9]:
####fit von log reg

lr = LogisticRegression(max_iter=10000,random_state=70)
lr.fit(X_train, y_train)
lr.score(X_train,y_train)

0.8490262920731946

In [10]:
### predic

y_pred = lr.predict(X_test)

In [11]:
### score für unsere trainingsdaten

score_print(y_train,lr.predict(X_train))

[[21668     0]
 [ 3853     0]]
              precision    recall  f1-score   support

           0       0.85      1.00      0.92     21668
           1       0.00      0.00      0.00      3853

    accuracy                           0.85     25521
   macro avg       0.42      0.50      0.46     25521
weighted avg       0.72      0.85      0.78     25521

/Users/kli/neuefische/13_Decision_trees/ds-ensemble-methods/.venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/kli/neuefische/13_Decision_trees/ds-ensemble-methods/.venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` p

In [12]:
#####evu

score_print(y_test,y_pred)

[[14445     0]
 [ 2569     0]]
              precision    recall  f1-score   support

           0       0.85      1.00      0.92     14445
           1       0.00      0.00      0.00      2569

    accuracy                           0.85     17014
   macro avg       0.42      0.50      0.46     17014
weighted avg       0.72      0.85      0.78     17014

/Users/kli/neuefische/13_Decision_trees/ds-ensemble-methods/.venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/kli/neuefische/13_Decision_trees/ds-ensemble-methods/.venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` p

In [13]:
######## DecisionTree
dtc = DecisionTreeClassifier(max_depth=10, min_samples_split=5, random_state=70)
dtc.fit(X_train,y_train)
y_pred_dtc=dtc.predict(X_test)
score_print(y_test,y_pred_dtc)

[[14109   336]
 [ 2433   136]]
              precision    recall  f1-score   support

           0       0.85      0.98      0.91     14445
           1       0.29      0.05      0.09      2569

    accuracy                           0.84     17014
   macro avg       0.57      0.51      0.50     17014
weighted avg       0.77      0.84      0.79     17014



In [14]:
### score auf unsere trainingdaten


score_print(y_train,dtc.predict(X_train))

[[21568   100]
 [ 3293   560]]
              precision    recall  f1-score   support

           0       0.87      1.00      0.93     21668
           1       0.85      0.15      0.25      3853

    accuracy                           0.87     25521
   macro avg       0.86      0.57      0.59     25521
weighted avg       0.86      0.87      0.82     25521



In [15]:
X_train.shape

(25521, 40)

In [56]:
y_train.shape

(25521,)

In [57]:
####LinearSVC

In [58]:
svc= LinearSVC(random_state=70)
svc.fit(X_train,y_train)
y_pred_svc=dtc.predict(X_test)
score_print(y_test,y_pred_svc)

[[5408 1314   22    1]
 [2058 5343  207    0]
 [ 254  769 1448   27]
 [   7   21   51   84]]
              precision    recall  f1-score   support

          10       0.70      0.80      0.75      6745
          15       0.72      0.70      0.71      7608
          20       0.84      0.58      0.69      2498
          25       0.75      0.52      0.61       163

    accuracy                           0.72     17014
   macro avg       0.75      0.65      0.69     17014
weighted avg       0.73      0.72      0.72     17014

/Users/kli/neuefische/13_Decision_trees/ds-ensemble-methods/.venv/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [59]:
########## score auf unsere trainingdaten


score_print(y_train,svc.predict(X_train))

[[10116     0     0     0]
 [11413     0     0     0]
 [ 3747     0     0     0]
 [  245     0     0     0]]
              precision    recall  f1-score   support

          10       0.40      1.00      0.57     10116
          15       0.00      0.00      0.00     11413
          20       0.00      0.00      0.00      3747
          25       0.00      0.00      0.00       245

    accuracy                           0.40     25521
   macro avg       0.10      0.25      0.14     25521
weighted avg       0.16      0.40      0.23     25521

/Users/kli/neuefische/13_Decision_trees/ds-ensemble-methods/.venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/kli/neuefische/13_Decision_trees/ds-ensemble-methods/.venv/lib/python3.8/site-packa

In [60]:
#######   KNeighborsClassifier

In [61]:
knn=KNeighborsClassifier(n_neighbors=5)

knn.fit(X_train,y_train)
y_pred_knn=knn.predict(X_test)
score_print(y_test,y_pred_knn)

[[4279 2366   99    1]
 [3158 4021  425    4]
 [ 730 1291  464   13]
 [  28   77   57    1]]
              precision    recall  f1-score   support

          10       0.52      0.63      0.57      6745
          15       0.52      0.53      0.52      7608
          20       0.44      0.19      0.26      2498
          25       0.05      0.01      0.01       163

    accuracy                           0.52     17014
   macro avg       0.38      0.34      0.34     17014
weighted avg       0.50      0.52      0.50     17014



In [62]:
########## score auf unsere trainingdaten


score_print(y_train,knn.predict(X_train))

[[7809 2215   92    0]
 [3088 7943  377    5]
 [1072 1514 1155    6]
 [  38  119   70   18]]
              precision    recall  f1-score   support

          10       0.65      0.77      0.71     10116
          15       0.67      0.70      0.68     11413
          20       0.68      0.31      0.42      3747
          25       0.62      0.07      0.13       245

    accuracy                           0.66     25521
   macro avg       0.66      0.46      0.49     25521
weighted avg       0.67      0.66      0.65     25521



In [63]:
###### svm

clf = svm.SVC(kernel='rbf')
#clf.fit(X_train,y_train)
#y_pred_clf=clf.predict(X_test)
#score_print(y_test,y_pred_clf)

In [64]:
########## score auf unsere trainingdaten


#score_print(y_train,clf.predict(X_train))

In [65]:
##### RandomForestClassifier

rfc= RandomForestClassifier(max_depth=5,min_samples_split=20)
rfc.fit(X_train,y_train)
y_pred_rfc=rfc.predict(X_test)
score_print(y_test,y_pred_rfc)

[[4334 2411    0    0]
 [2009 5594    5    0]
 [ 268 2144   86    0]
 [   4  133   26    0]]
              precision    recall  f1-score   support

          10       0.66      0.64      0.65      6745
          15       0.54      0.74      0.63      7608
          20       0.74      0.03      0.07      2498
          25       0.00      0.00      0.00       163

    accuracy                           0.59     17014
   macro avg       0.48      0.35      0.33     17014
weighted avg       0.61      0.59      0.55     17014

/Users/kli/neuefische/13_Decision_trees/ds-ensemble-methods/.venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/kli/neuefische/13_Decision_trees/ds-ensemble-methods/.venv/lib/python3.8/site-packages/sklearn/metr

In [66]:
########## score auf unsere trainingdaten


score_print(y_train,rfc.predict(X_train))

[[6623 3493    0    0]
 [2906 8506    1    0]
 [ 447 3148  152    0]
 [   7  192   46    0]]
              precision    recall  f1-score   support

          10       0.66      0.65      0.66     10116
          15       0.55      0.75      0.64     11413
          20       0.76      0.04      0.08      3747
          25       0.00      0.00      0.00       245

    accuracy                           0.60     25521
   macro avg       0.50      0.36      0.34     25521
weighted avg       0.62      0.60      0.56     25521

/Users/kli/neuefische/13_Decision_trees/ds-ensemble-methods/.venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/kli/neuefische/13_Decision_trees/ds-ensemble-methods/.venv/lib/python3.8/site-packages/sklearn/metr

In [67]:
%store X_train X_test y_test y_train score_print

Stored 'X_train' (DataFrame)
Stored 'X_test' (DataFrame)
Stored 'y_test' (Series)
Stored 'y_train' (Series)
Proper storage of interactively declared classes (or instances
of those classes) is not possible! Only instances
of classes in real modules on file system can be %store'd.

